<a href="https://colab.research.google.com/github/xBosse/notebooks/blob/main/Modell_aufbauen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
! pip install datasets transformers seqeval
import transformers, datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# LOAD DATASET

In [ ]:
from datasets import load_dataset, load_metric, ClassLabel, Value, Features

import datasets

#DATENSATZ
train_url = "https://raw.githubusercontent.com/xBosse/datasets/main/train_shata14.txt"

val_url = "https://raw.githubusercontent.com/xBosse/datasets/main/val_shata14.txt"

test_url = "https://raw.githubusercontent.com/xBosse/datasets/main/test_steps16.txt"


# eigener Code

classFeatures = datasets.Features({
    'words': datasets.Sequence(Value("string")),
    'subj': datasets.Sequence(ClassLabel(names=['O', 'B-SUBJ', 'I-SUBJ']))
})

# eigener Parameter

datasets = load_dataset('json', data_files = { 'train': train_url, 'validation': val_url, 'test': test_url}, features=classFeatures)

task = "subj_v6_7Epoch_v3" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "bert-base-german-cased"

# bert-base-german-cased 

batch_size = 16

Using custom data configuration default-68ea738c7c1a98fd
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-68ea738c7c1a98fd/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


  0%|          | 0/3 [00:00<?, ?it/s]

# Preprocess data / Tokenizer


In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [ ]:
label_all_tokens = True

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["words"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"subj"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)

Loading cached processed dataset at /root/.cache/huggingface/datasets/json/default-68ea738c7c1a98fd/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b/cache-77cdb4043b666685.arrow


  0%|          | 0/1 [00:00<?, ?ba/s]

Loading cached processed dataset at /root/.cache/huggingface/datasets/json/default-68ea738c7c1a98fd/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b/cache-427887343fcd05f4.arrow


# Fine Tuning

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

#LABELS # eigener Code
#label_list = ['O', 'B-SUBJ', 'I-SUBJ']
label_list = datasets["train"].features[f"subj"].feature.names

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-b

In [ ]:


model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=7,                           ######### hyperparameter - epoch ##########
    weight_decay=0.01,
    push_to_hub=True,
)

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

metric = load_metric("seqeval")

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
! git config --global credential.helper store

from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token


In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Cloning https://huggingface.co/tbosse/bert-base-german-cased-finetuned-subj_v6_7Epoch_v3 into local empty directory.


In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: subj, words. If subj, words are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 520
  Num Epochs = 7
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 231


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.328146,0.665595,0.591429,0.626324,0.862327
2,No log,0.262334,0.743976,0.705714,0.724340,0.894009
3,No log,0.245983,0.753582,0.751429,0.752504,0.906682
4,No log,0.243965,0.777778,0.760000,0.768786,0.912442
5,No log,0.258151,0.772334,0.765714,0.769010,0.910714
6,No log,0.268110,0.769014,0.780000,0.774468,0.911866
7,No log,0.273182,0.765363,0.782857,0.774011,0.911866


The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: subj, words. If subj, words are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 58
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: subj, words. If subj, words are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 58
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: subj, words. If subj, words are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 58
  Batch size 

TrainOutput(global_step=231, training_loss=0.1861489861558526, metrics={'train_runtime': 2250.3551, 'train_samples_per_second': 1.618, 'train_steps_per_second': 0.103, 'total_flos': 120671199441216.0, 'train_loss': 0.1861489861558526, 'epoch': 7.0})

In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: subj, words. If subj, words are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 58
  Batch size = 16


{'epoch': 7.0,
 'eval_accuracy': 0.9118663594470046,
 'eval_f1': 0.7740112994350282,
 'eval_loss': 0.2731815278530121,
 'eval_precision': 0.7653631284916201,
 'eval_recall': 0.7828571428571428,
 'eval_runtime': 10.8398,
 'eval_samples_per_second': 5.351,
 'eval_steps_per_second': 0.369}

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: subj, words. If subj, words are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 58
  Batch size = 16


{'SUBJ': {'f1': 0.7740112994350282,
  'number': 350,
  'precision': 0.7653631284916201,
  'recall': 0.7828571428571428},
 'overall_accuracy': 0.9118663594470046,
 'overall_f1': 0.7740112994350282,
 'overall_precision': 0.7653631284916201,
 'overall_recall': 0.7828571428571428}

In [ ]:
trainer.push_to_hub()

Saving model checkpoint to bert-base-german-cased-finetuned-subj_v6_7Epoch_v3
Configuration saved in bert-base-german-cased-finetuned-subj_v6_7Epoch_v3/config.json
Model weights saved in bert-base-german-cased-finetuned-subj_v6_7Epoch_v3/pytorch_model.bin
tokenizer config file saved in bert-base-german-cased-finetuned-subj_v6_7Epoch_v3/tokenizer_config.json
Special tokens file saved in bert-base-german-cased-finetuned-subj_v6_7Epoch_v3/special_tokens_map.json


Upload file pytorch_model.bin:   0%|          | 3.34k/414M [00:00<?, ?B/s]

Upload file training_args.bin: 100%|##########| 3.17k/3.17k [00:00<?, ?B/s]

Upload file runs/May25_18-16-02_ac72b39de180/events.out.tfevents.1653504848.ac72b39de180.781.2: 100%|#########…

Upload file runs/May25_18-16-02_ac72b39de180/1653502586.9204276/events.out.tfevents.1653502586.ac72b39de180.78…

Upload file runs/May25_18-16-02_ac72b39de180/events.out.tfevents.1653502586.ac72b39de180.781.0:  46%|####6    …

remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/tbosse/bert-base-german-cased-finetuned-subj_v6_7Epoch_v3
   781c1b4..4771457  main -> main

Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Token Classification', 'type': 'token-classification'}, 'metrics': [{'name': 'Precision', 'type': 'precision', 'value': 0.7653631284916201}, {'name': 'Recall', 'type': 'recall', 'value': 0.7828571428571428}, {'name': 'F1', 'type': 'f1', 'value': 0.7740112994350282}, {'name': 'Accuracy', 'type': 'accuracy', 'value': 0.9118663594470046}]}
remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/tbosse/bert-base-german-cased-finetuned-subj_v6_7Epoch_v3
   4771457..73b1feb  main -> main



'https://huggingface.co/tbosse/bert-base-german-cased-finetuned-subj_v6_7Epoch_v3/commit/4771457ec4347c172f015a889f7ef18df9ebab43'

# Model Checkpoint

In [ ]:
from transformers import AutoTokenizer,AutoModelForTokenClassification

model_name = "tbosse/bert-base-german-cased-finetuned-subj_preTrained_with_noisyData"

model = AutoModelForTokenClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
from transformers import AutoTokenizer,AutoModelForTokenClassification

model_name = "tbosse/bert-base-german-cased-finetuned-subj_preTrained_with_noisyData"

model = AutoModelForTokenClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, padding=True, truncation=True, max_length=512)

from transformers import pipeline

classifier = pipeline("token-classification", model=model, tokenizer=tokenizer) # , padding=True, truncation=True, max_length=512
#result_test = classifier("Ich mag keine Birnen.")

In [ ]:

classifier("Ich mag keine Birnen.")


[{'end': 3,
  'entity': 'LABEL_0',
  'index': 1,
  'score': 0.9998534,
  'start': 0,
  'word': 'Ich'},
 {'end': 7,
  'entity': 'LABEL_0',
  'index': 2,
  'score': 0.99266934,
  'start': 4,
  'word': 'mag'},
 {'end': 13,
  'entity': 'LABEL_0',
  'index': 3,
  'score': 0.99948525,
  'start': 8,
  'word': 'keine'},
 {'end': 17,
  'entity': 'LABEL_0',
  'index': 4,
  'score': 0.96852255,
  'start': 14,
  'word': 'Bir'},
 {'end': 20,
  'entity': 'LABEL_0',
  'index': 5,
  'score': 0.97981334,
  'start': 17,
  'word': '##nen'},
 {'end': 21,
  'entity': 'LABEL_0',
  'index': 6,
  'score': 0.99904555,
  'start': 20,
  'word': '.'}]